# Sample Cloud AI Platform Notebook: Predicting Visitor Behaviour
Predict if a visitor will add items to the cart using their browsing session data

# Import

In [ ]:
import os
import time
from datetime import datetime

import pandas as pd
import numpy as np

from tqdm import tqdm

import tensorflow as tf
from tensorflow import feature_column
import tensorflow_io as tfio

# Load and plot some data


Let's use the Google Cloud datawharehouse [BigQuery](https://cloud.google.com/bigquery/docs/introduction) to query the data.  
The BigQuery client library provides a cell magic ```%%bigquery``` which runs a SQL query and returns the results as a Pandas DataFrame.  
Use the cell magic to query a sample of data and save the results in the ```train_df``` DataFrame:

In [ ]:
%%bigquery train_df
SELECT 
  deviceCategory,
  totalPageViews,
  addedToCart
FROM
  `kaggleworkshops.google_analytics_sample_eu.train`
WHERE date BETWEEN TIMESTAMP("2016-08-01")
  AND TIMESTAMP("2016-08-31")

Show the first few rows of the DataFrame:

In [ ]:
train_df.head()

Show the DataFrame details:

In [ ]:
train_df.describe()

Plot the data by ```deviceCategory``` and sort by ```totalPageViews```:

In [ ]:
train_df.groupby('deviceCategory').sum().sort_values('totalPageViews', ascending=False).head(10).plot(kind='bar', logy=True)

# Local training

First, let's try to train the model locally (from within the notebook).

## Create a Google Cloud Storage bucket

We need a way to centrally store and share data across services.  
Let's use [Google Cloud Storage](https://cloud.google.com/storage) which is the blob storage service from Google Cloud.

```gsutil``` is a command line tool for Google Cloud Storage.  
In Google Cloud Storage, URI are in the format ```gs://bucket/folder/file``` .  
Use ```gsutil mb gs://YourBucketName``` to create a Google Cloud Storage bucket.

Your bucket name must be **globally** unique and must contain only lowercase letters, numbers, dashes, underscores, and dots.  
**!!! Change the bucket name below with your own!!!**

In [ ]:
# Prepare Google Cloud Storage directory to save logs and model
BUCKET_NAME = 'ml-competition-mpe-team' # Create your own unique bucket name
!gsutil mb -l EU gs://{BUCKET_NAME}

Use ```gsutil ls gs://YourBucketName``` to list your bucket and make sure it has been correctly created.  
Expect no output if the bucket is correctly created.

In [ ]:
!gsutil ls gs://{BUCKET_NAME}

## Create your trainer package structure

Now let's create your trainer package structure.

In [ ]:
# Create the Trainer package structure
!mkdir ./trainer
!touch ./trainer/__init__.py

**!!! Change the bucket name below with your own!!!**

In [ ]:
%%writefile ./trainer/task.py

# Imports
import os
import pandas as pd
import tensorflow as tf
from google.cloud import bigquery
from sklearn.model_selection import train_test_split

# Declare constants
BUCKET_NAME = 'ml-competition-mpe-team'  # change to YOUR bucket name
JOB_DIR = f'gs://{BUCKET_NAME}/latest_model' 
CATEGORICAL_TYPES = {'deviceCategory': pd.api.types.CategoricalDtype(
                                ['desktop', 'mobile', 'tablet']
                            )}
TARGET_COLUMN = 'addedToCart'
QUERY = '''SELECT 
  deviceCategory,
  totalPageViews,
  addedToCart
FROM
  `kaggleworkshops.google_analytics_sample.train`
WHERE date BETWEEN TIMESTAMP("2016-08-01")
  AND TIMESTAMP("2016-08-31")'''

BATCH_SIZE = 128
NUM_EPOCHS = 5
LEARNING_RATE = 0.001

# Read the data from BigQuery
client = bigquery.Client() 
query_job = client.query(QUERY)
data_df = query_job.to_dataframe()  # you can read from other sources to pandas DataFrame
print(f'First rows for the raw dataset: \n{data_df.head()}')

# Convert integer valued (numeric) columns to floating point
numeric_columns = data_df.select_dtypes(['int64']).columns
data_df[numeric_columns] = data_df[numeric_columns].astype('float32')

# Convert categorical columns to numeric
cat_columns = data_df.select_dtypes(['object']).columns
data_df[cat_columns] = data_df[cat_columns].astype('category')
data_df[cat_columns] = data_df[cat_columns].apply(lambda x: x.astype(
        CATEGORICAL_TYPES[x.name]))
data_df[cat_columns] = data_df[cat_columns].apply(lambda x: x.cat.codes)
print(f'First rows for the transformed dataset: \n{data_df.head()}')

# Train/Val split
train_df, val_df = train_test_split(data_df, train_size=0.8)
train_target = train_df.pop(TARGET_COLUMN)
val_target = val_df.pop(TARGET_COLUMN)
num_train_examples = len(train_df)
num_val_examples = len(val_df)

# *Possible improvements*: add standartization for numeric values to range [-1; 1], categories to one-hot encoded

# Creata tensorflow dataset object
dataset_train = (tf.data.Dataset
                 .from_tensor_slices((train_df.to_dict('list'), train_target))
                 .shuffle(buffer_size=BATCH_SIZE*4)
                 .repeat()
                 .batch(BATCH_SIZE))
                 
dataset_val = (tf.data.Dataset
                 .from_tensor_slices((val_df.to_dict('list'), val_target))
                 .repeat()
                 .batch(BATCH_SIZE))  # No shuffle

print(f'One batch of the train data:\n {next(iter(dataset_train))}')

# Prepare named inputs for our model
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in train_df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

# Define model's architecture
x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)

# Build the model and compile it
model_func = tf.keras.Model(inputs=inputs, outputs=output)
model_func.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
                   loss='binary_crossentropy',
                   metrics=['accuracy', tf.keras.metrics.AUC()])

# Train the model
history = model_func.fit(dataset_train, 
                          epochs=NUM_EPOCHS, 
                          steps_per_epoch=int(num_train_examples/BATCH_SIZE), 
                          validation_data=dataset_val, 
                          validation_steps=int(num_train_examples/BATCH_SIZE), 
                          verbose=1)

model_func.save(f'{JOB_DIR}/export/')

In [ ]:
%%writefile ./setup.py
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['scikit-learn', 'pandas']

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='My super training application package.'
)

## Run a local training task
This is a good test before your will try to train the model in the cloud

Many commands we are going to use accept a parameter for setting a region.  
A region is a group of Google Cloud data centers used to run computing tasks.  
To reduce latency, let's set a variable with a close by data center:

In [ ]:
REGION = 'europe-west1'

For interacting with Cloud AI Platform we are going to use the [gcloud](https://cloud.google.com/sdk/gcloud) command line tool.  
Gcloud also provides [properties](https://cloud.google.com/sdk/docs/properties) used by other services.  
Let's set the __ml_engine/local_python__ property so AI Platform knows which Python version to use for local training:

In [ ]:
# Explicitly tell `gcloud ai-platform local train` to use Python 3 
!gcloud config set ml_engine/local_python $(which python3)

For submitting a local training job to AI Platform you need to provide:
* A directory to store the model and logs: here we are going to use our Google Cloud Storage bucket we created earlier
* The path to your trainer package
* The name of your trainer module

In [ ]:
# Define a timestamped job name
JOB_NAME = f"training_job_{datetime.now().strftime('%Y_%m_%d_%H%M%S')}"; print(JOB_NAME)
JOB_DIR = f'gs://{BUCKET_NAME}/{JOB_NAME}'; print(JOB_DIR)

Let's run our local training job with the gcloud command ```gcloud ai-platform local train```.

In [ ]:
# Run the localtraining job
! gcloud ai-platform local train \
  --job-dir $JOB_DIR \
  --package-path ./trainer \
  --module-name trainer.task 

Your model has been saved to your Google Cloud Storage bucket.

In [ ]:
!gsutil ls gs://{BUCKET_NAME}

# AI Platform training

Train your model in the cloud.  
This can help when you'll need more compute power, run your training for a long periods of time or try several trainings in parallel with hyperparameters search.

When training through AI Platform you need a few more parameters:
* __Region__: the region used by AI Platform for training
* __Runtime version__: the AI Platform version you want to use
* __Python version__: the Python version used by your package
* __Scale tier__: define which compute power will be used (GPU, TPU, number of machines, ...), more details in [this documentation](https://cloud.google.com/ai-platform/training/docs/machine-types)

Let's submit a training job with ```gcloud ai-platform jobs submit training``` with a basic configuration (only 1 machine, no GPU, no TPU):

In [ ]:
# Submit the training job
! gcloud ai-platform jobs submit training $JOB_NAME \
  --job-dir $JOB_DIR \
  --package-path ./trainer \
  --module-name trainer.task \
  --region $REGION \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --scale-tier basic 

The training job is a long running operation.  
You can use ```gcloud ai-platform jobs describe``` to get the status of the job:

In [ ]:
! gcloud ai-platform jobs describe $JOB_NAME

Your model has been saved to your Google Cloud Storage bucket.

In [ ]:
!gsutil ls gs://{BUCKET_NAME}

# AI Platform deployment

Now that you have trained your model, it's time to make it available for serving predictions.  
Google [AI Platform Prediction](https://cloud.google.com/ai-platform/prediction/docs) lets you do just that very easily.

Let's use ```gsutil ls``` to list your model's file from the Cloud Storage Bucket:

In [ ]:
LATEST_MODEL_DIR = f'gs://{BUCKET_NAME}/latest_model/export' 
!gsutil ls -lh $LATEST_MODEL_DIR

Set a **name** and a **version** for this model:

In [ ]:
MODEL_NAME = 'clf_add_to_cart' # Choose your own model name
MODEL_VERSION = 'v1' # Make sure to increase version when deploying a new version of the same model

Let's use ```gcloud ai-platform models create```to create a new model:

In [ ]:
# create a model object at AI Platform first
! gcloud ai-platform models create $MODEL_NAME \
  --regions $REGION

Now that we have the model available in AI Platform, let's create the first version of this model.  
We need to point AI Platform to our model in Google Cloud Storage.

In [ ]:
# Create model version based on that SavedModel directory
! gcloud ai-platform versions create $MODEL_VERSION \
  --model $MODEL_NAME \
  --region global \
  --runtime-version 2.1 \
  --python-version 3.7 \
  --framework tensorflow \
  --origin $LATEST_MODEL_DIR

# Prediction with AI Platform (from a csv file)
At this point we have trained a model and made the model available for serving predictions thanks to AI Platform.  
Let's get some predictions from this model.

## Get the test data
Let's grab some fresh data to generate predictions on!

In [ ]:
%%bigquery test_df
SELECT 
  CONCAT(fullVisitorId, CAST(visitStartTime as string)) as sessionId, 
  deviceCategory,
  totalPageViews
FROM
  `kaggleworkshops.google_analytics_sample_eu.test`

In [ ]:
test_df.head()

In [ ]:
test_df.describe()

In [ ]:
test_df.groupby('deviceCategory').sum().sort_values('totalPageViews', ascending=False).head(10).plot(kind='bar', logy=True)

In [ ]:
# Need to avoid "serving skew"! 
# Preprocess test data the same way as we did for training
CATEGORICAL_TYPES = {'deviceCategory': pd.api.types.CategoricalDtype(
                                ['desktop', 'mobile', 'tablet']
                    )}

# Convert integer valued (numeric) columns to floating point
numeric_columns = test_df.select_dtypes(['int64']).columns
test_df[numeric_columns] = test_df[numeric_columns].astype('float32')

# Convert categorical columns to numeric
cat_columns = list(CATEGORICAL_TYPES.keys())
test_df[cat_columns] = test_df[cat_columns].astype('category')
test_df[cat_columns] = test_df[cat_columns].apply(lambda x: x.astype(
        CATEGORICAL_TYPES[x.name]))
test_df[cat_columns] = test_df[cat_columns].apply(lambda x: x.cat.codes)
print(f'First rows for the transformed dataset: \n{test_df.head()}')

### Prepare prediction input file
The `gcloud` command-line tool accepts newline-delimited JSON for online
prediction, and this particular Keras model expects a flat list of
numbers for each input example.

AI Platform requires a different format when you make online prediction requests to the REST API without using the `gcloud` tool. The way you structure
your model may also change how you must format data for prediction. Learn more
about [formatting data for online
prediction](https://cloud.google.com/ml-engine/docs/tensorflow/prediction-overview#prediction_input_data).

```
{"instances": [
  {"deviceCategory": 1, "totalPageViews": 10},
  {"deviceCategory": 2, "totalPageViews": 1}
]}
```

Test first on a few samples:

In [ ]:
# Export the prediction input to a JSON file in the format accepted by AI Platform
import json

prediction_dict_sample = test_df.drop('sessionId', axis=1)[:5].to_dict('records')

with open('prediction_input.json', 'w') as json_file:
    json.dump({'instances': prediction_dict_sample}, json_file, indent=' ')

! cat prediction_input.json

### Test predictions on few samples

Use ```gcloud ai-platform predict``` to generate predictions from your model:

In [ ]:
! gcloud ai-platform predict \
  --model $MODEL_NAME \
  --region global \
  --version $MODEL_VERSION \
  --json-request prediction_input.json

### Online predictions on the whole dataset

Let's now get predictions for the whole dataset.

In [ ]:
# Helper copied from the AI Platform console
import googleapiclient.discovery

def predict_json(project, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

We need to provide our Project ID to the online predictions service.  
Project ID is a unique identifier for the Google Cloud environment you are currently using.  
Let's use ```gcloud config get-value project``` to get this property from gcloud:

In [ ]:
PROJECT_ID = !gcloud config get-value project

In [ ]:
PROJECT_ID = PROJECT_ID.get_nlstr()

In [ ]:
from functools import partial

In [ ]:
get_predictions = partial(
    predict_json,
    project=PROJECT_ID, 
    model=MODEL_NAME, 
    version=MODEL_VERSION
)

In [ ]:
BATCH_SIZE = 1024

In [ ]:
num_batches = len(test_df)//BATCH_SIZE; num_batches

In [ ]:
prediction_scores = []
session_ids = test_df.pop('sessionId')

for i in tqdm(range(num_batches+1), total=num_batches, position=0):
    batch_df = test_df.iloc[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:]
    pred = get_predictions(instances=batch_df.to_dict('records'))
    pred = [p['dense_1'][0] for p in pred]
    prediction_scores.extend(pred)

In [ ]:
prediction_scores[:5]

# Export results as a csv file

Now that we have our predictions, let's export them as a csv file.

In [ ]:
predictions = pd.DataFrame(
    {'sessionId': session_ids, 
     'addedToCart':prediction_scores}, 
    columns=['sessionId', 'addedToCart'])

In [ ]:
predictions.info()

In [ ]:
predictions.to_csv('predictions.csv', index=False)

In [ ]:
!head predictions.csv

# Clean up

Delete all versions and all models:

In [ ]:
import time
import googleapiclient.discovery

service = googleapiclient.discovery.build('ml', 'v1')

project = !gcloud config get-value project
project = project.get_nlstr()

def get_models(project):
    response = service.projects().models().list(
        parent = 'projects/{}'.format(project)
    ).execute()
    
    return response["models"]

def get_versions(model):
    response = service.projects().models().versions().list(
        parent=model
    ).execute()
    
    return response["versions"]

def delete_version(version):
    print("Deleting version: ", version["name"])
    
    response = service.projects().models().versions().delete(
        name=version["name"]
    ).execute()
    
    if "error" in response:
        print(error)
    
    return response["name"]

def delete_model(model):
    print("Deleting model: ", model["name"])
    
    response = service.projects().models().delete(
        name=model["name"]
    ).execute()
    
    if "error" in response:
        print(error)

def is_version_deleted(operation):
    print("Checking status for operation: ", operation)
    
    response = service.projects().operations().get(
        name=operation
    ).execute()
    
    print(response)
    if "done" in response:
        return True
    else:
        return False

models = get_models(project)

default_version_deletions = []
for model in models:
    print('Model: ', model["name"])
    versions = get_versions(model["name"])
    deletions_in_progress = []
    for version in versions:
        # Delete non default versions
        if "isDefault" not in version:
            versions.remove(version)
            deletions_in_progress.append(delete_version(version))
    while len(deletions_in_progress) > 0:
        # Try again in 5s
        print("Waiting 5s")
        time.sleep(5)
        for deletion_in_progress in deletions_in_progress:
            if is_version_deleted(deletion_in_progress):
                print("Deletion completed: ", deletion_in_progress)
                deletions_in_progress.remove(deletion_in_progress)
    # When all default versions are deleted, remove the default version
    default_version_deletions.append(delete_version(versions[0]))
while len(default_version_deletions) > 0:
    # Try again in 5s
    print("Waiting 5s")
    time.sleep(5)
    for default_version_deletion in default_version_deletions:
        if is_version_deleted(default_version_deletion):
            print("Deletion completed: ", default_version_deletion)
            default_version_deletions.remove(default_version_deletion)
# All versions deleted, now delete the model
for model in models:
    delete_model(model)

Delete the bucket:

In [ ]:
# Delete your bucket
!gsutil rm -r gs://{BUCKET_NAME}